## Bubble sheet scanner and grader using OMR, Python and OpenCV

In [15]:
import numpy as np
import imutils
from imutils import contours
import cv2

In [2]:
# To order points
def order_points(pts):
    rect = np.zeros((4,2),dtype = "float32")
    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    
    return rect

In [3]:
#To transform from 3D to 2D
def four_point_transform(image, pts):
    # obtain a consistent order of the points and unpack them
    # individually
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    # compute the width maximum distance between bottom-right and bottom-left
    # x-coordiates or the top-right and top-left
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    # compute the height maximum distance between the top-right and bottom-right
    # y-coordinates or the top-left and bottom-left y-coordinates
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    # now that we have the dimensions of the new image, construct
    # the set of destination points to obtain a "birds eye view"
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")
    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    # return the warped image
    return warped

# Answer Key

In [4]:
answer_key = {0:1,1:4,2:0,3:3,4:1}

# Image Reading

In [5]:
image = cv2.imread("omr_test_01.png")
orig = image.copy()
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
# blurring to remove noise
blurred = cv2.GaussianBlur(gray,(5,5),0)
edged = cv2.Canny(blurred,75,150)
cv2.imshow("Edged",edged)
cv2.waitKey(0)
cv2.destroyWindow("Edged")

# Extracting our Answer sheet from Background

In [6]:
# finding contours
cnts = cv2.findContours(edged.copy(),cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
docCnt = None
# Detecting largest rectangle(our answer sheet)
if len(cnts) > 0:
    cnts = sorted(cnts,key=cv2.contourArea,reverse=True)
    for c in cnts:
        peri = cv2.arcLength(c,True)
        approx = cv2.approxPolyDP(c,0.02*peri,True)
        if len(approx) == 4:
            docCont = approx
            break

#changing the view
paper = four_point_transform(image,docCont.reshape(4,2))
wraped = four_point_transform(gray,docCont.reshape(4,2))
paper_copy = paper.copy()
cv2.imshow("Paper",paper)
cv2.imshow("Wraped",wraped)
cv2.waitKey(0)
cv2.destroyWindow("Paper") or cv2.destroyWindow("Wraped")

# Applying Theshold

In [7]:
tresh = cv2.threshold(wraped,0,255,
                     cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
cv2.imshow("Thresh",tresh)
cv2.waitKey(0)
cv2.destroyWindow("Thresh")

# Finding Contours for Circles(Bubbles)
<a href = "https://docs.opencv.org/3.4/d9/d8b/tutorial_py_contours_hierarchy.html">for Contour Hierarchy</a>

In [8]:
#Retriving externel contours
cnts = cv2.findContours(tresh.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
questionCnts = []
for c in cnts:
    (x,y,w,h) = cv2.boundingRect(c)
    ar = w/float(h)
    if w>=20 and h>=20 and ar>=0.9 and ar<=1.1:
        questionCnts.append(c)

for i in questionCnts:
    cv2.drawContours(paper_copy,[i],-1,(0,0,255),4)

In [9]:
cv2.imshow("Questions",paper_copy)
cv2.waitKey(0)
cv2.destroyWindow("Questions")

# Finding Correct,Skipped,Multiple attempted answers

In [10]:
# sort the contours to obtain row wise list of contours
# from the contours we obtained above
questionCnts = contours.sort_contours(questionCnts,
                                      method = "top-to-bottom")[0]
correct = 0
skipped = 0
multiple = 0

# sort the contours of each row to obtain
# contoures of column in-order
for (q,i) in enumerate(np.arange(0,len(questionCnts),5)):
    cnts = contours.sort_contours(questionCnts[i:i+5])[0]
    bubbled = None
    count = 0
    #Loop through each contour and find if it is bubbled or not
    #500 is the threshold pixel value of a bubbled one
    for (j,c) in enumerate(cnts):
        mask = np.zeros(tresh.shape,dtype="uint8")
        cv2.drawContours(mask,[c],-1,255,-1)
        mask = cv2.bitwise_and(tresh, tresh, mask=mask)
        total = cv2.countNonZero(mask)
        if total >= 500:
            count +=1 
        if bubbled is None or total > bubbled[0]:
            bubbled = (total,j)
        color = (0,0,255)
    k = answer_key[q]
    if count == 0:
        skipped+=1
    if k == bubbled[1] and count == 1:
        color = (0,255,0)
        correct += 1
    if count>1:
        multiple +=1
        
    cv2.drawContours(paper,[cnts[k]],-1,color,3)
print("Corrected",correct)
print("Skipped",skipped)
print("Multiple",multiple)

Corrected 4
Skipped 0
Multiple 0


In [14]:
percentage = (correct/5)*100
cv2.putText(paper,"{:.2f}%".format(percentage),(10,25),
           cv2.FONT_HERSHEY_SIMPLEX,0.7,(255,0,0),2)
cv2.imshow("Paper",paper)
cv2.waitKey(0)
cv2.destroyWindow("Paper")

<a href = "https://www.pyimagesearch.com/2016/10/03/bubble-sheet-multiple-choice-scanner-and-test-grader-using-omr-python-and-opencv/">For detailed Explanation</a>